In [13]:
import numpy as np
import spatialpower.tissue_generation.assign_labels as assign_labels
import spatialpower.tissue_generation.visualization as viz
import spatialpower.neighborhoods.permutationtest as perm_test

import networkx as nx 

In [ ]:
%%bash 

python spatialpower/tissue_generation/random_circle_packing.py -x 1000 -y 1000 -o sample_results 

# see python spatialpower/tissue_generation/random_circle_packing.py --help for full options

### Load circle packings

In [2]:
A = np.load('./sample_results/A.npy')
C =  np.load('./sample_results/C.npy')

### Set parameters

In [3]:
cell_type_probabilities = np.array([0.7, 0.1, 0.1, 0.1])


neighborhood_probabilities = np.array(([0.60, 0.13, 0.13, 0.14],
                                        [0.13, 0.29, 0.29, 0.29],
                                        [0.13, 0.29, 0.29, 0.29],
                                        [0.14, 0.29, 0.29, 0.28]))


### Optimized Assignment

Estimated runtime ~15 mins with no GPU

In [6]:
cell_assignments = assign_labels.optimize(A, cell_type_probabilities, neighborhood_probabilities, learning_rate=1e-5, iterations = 10)

102669.56
95909.34
89673.34
83920.75
constraint  160566.34
26444.54
26393.512
26347.734
26306.64
constraint  24031.094
44861.234
44855.277
44849.32
44843.383
constraint  23362.074
79455.195
79438.9
79422.61
79406.33
constraint  23038.674
143482.14
143433.55
143385.02
143336.45
constraint  22516.639
261517.2
261361.42
261205.9
261050.56
constraint  21655.24
473353.47
472833.53
472314.44
471795.8
constraint  20221.168
827157.25
825413.06
823671.2
821932.44
constraint  17873.21
1320851.0
1315265.9
1309698.6
1304151.9
constraint  14255.502
1705477.9
1690085.6
1674816.8
1659676.5
constraint  9479.71


In [7]:
def build_assignment_matrix(attribute_dict, n_cell_types):
    data = list(attribute_dict.items())
    data = np.array(data) # Assignment matrix
    
    B = np.zeros((data.shape[0],n_cell_types)) # Empty matrix
    
    for i in range(0, data.shape[0]):
        t = data[i,1]
        B[i,t] = 1
    
    return B 

In [10]:
B = cell_assignments.copy()

### Actual results

In [11]:
perm_test.calculate_neighborhood_distribution(A, B)

array([[0.50926333, 0.16338761, 0.16514334, 0.1644108 ],
       [0.51014847, 0.14834161, 0.15325385, 0.16741487],
       [0.51030556, 0.15429666, 0.16309762, 0.16614074],
       [0.49062609, 0.16007809, 0.16183156, 0.17577564]])

In [12]:
np.divide(np.sum(B, axis=0), np.sum(B))

array([0.51183653, 0.16371792, 0.16022925, 0.1642163 ])

In [14]:
position_dict = dict()
for i in range(0, C.shape[0]):
    position_dict[i] = C[i, :]

### Create visualization of tissue

In [17]:
viz.make_vor(1000, np.argmax(cell_assignments, axis=1), position_dict, 4, './sample_results/', 'test', [x for x in range(0, C.shape[0])])

### Heuristic assignment

Estimated runtime ~30s

In [18]:
graph = nx.from_numpy_array(A)

cell_assignments = assign_labels.heuristic_assignment(graph, cell_type_probabilities, neighborhood_probabilities, mode='graph', dim=1000, position_dict=position_dict, grid_size=50, revision_iters=100, n_swaps=25)

In [20]:
B = build_assignment_matrix(cell_assignments, n_cell_types=4)

### observed results


In [21]:
perm_test.calculate_neighborhood_distribution(A, B)

array([[0.56692352, 0.14563081, 0.13742473, 0.15002095],
       [0.44158258, 0.19238652, 0.18695437, 0.17907653],
       [0.43342818, 0.19954317, 0.17421409, 0.19281456],
       [0.44527197, 0.18153189, 0.17959374, 0.1936024 ]])

In [22]:
np.divide(np.sum(B, axis=0), np.sum(B))

array([0.51264542, 0.16590794, 0.15553869, 0.16590794])

### Visualize results

In [19]:
viz.make_vor(1000, cell_assignments, position_dict, 4, './sample_results/', 'test_heuristic', [x for x in range(0, C.shape[0])])